In [112]:
import pickle
import networkx as nx
import os
import numpy as np

In [7]:
#pickle_in = open("plans/10_1.pickle", "rb")
example_dict = nx.read_gpickle('plans/10_1.pickle')
example_dict

In [65]:
example_dict2 = nx.read_gpickle('plans/10_2.pickle')

In [40]:
example_dict.nodes[3137]

{'GEOID': '421012624',
 'area': 2.5217475641e-05,
 'democratic_votes': nan,
 'district': 1,
 'geometry': <shapely.geometry.polygon.Polygon at 0x150eb25198>,
 'population': 2,
 'republican_votes': nan}

In [ ]:
def district_connect(graph,n_districts):
    district_set = []
    uncon = []
    for x in range(n_districts):
        district_set.append([i for i in G if G.node[i]['district'] == x + 1]) 
    
    for x in range(len(district_set)):
         if nx.is_connected(nx.subgraph(G,district_set[x])) == False:
                return False
    return True

In [123]:
import numpy as np
def outcome_wins(graph,n_district):
    district_set = []
    for x in range(n_district):
        district_set.append([i for i in graph if graph.node[i]['district'] == x + 1])
    outcome = [[0,0]] * n_district
    for x in range(n_district):
        for node in district_set[x]:
            outcome[x][0] += graph.node[node]['democratic_votes']
            outcome[x][1] += graph.node[node]['republican_votes']
    dem_win = 0
    for x in range(len(outcome)):
        if outcome[x][0] > outcome[x][1]:
            dem_win += 1
    
    return dem_win
  
def get_dem_vote_percent(graph,n_district):
    district_set = []
    for x in range(n_district):
        district_set.append([i for i in graph if graph.nodes[i]['district'] == x + 1])
    outcome_dem = [0] * n_district
    outcome_rep = [0] * n_district
    for x in range(n_district):
        for node in district_set[x]:
            #print(graph.nodes[node]['democratic_votes'])
            #0 if math.isnan(graph.nodes[node]['democratic_votes'] else math.isnan(graph.nodes[node]['democratic_votes'])
            if str(graph.nodes[node]['democratic_votes']) != 'nan':
                outcome_dem[x] += float(graph.nodes[node]['democratic_votes'])
            if str(graph.nodes[node]['republican_votes']) != 'nan':
                outcome_rep[x] += float(graph.nodes[node]['republican_votes'])
    outcome_dem = np.array(outcome_dem)
    outcome_rep = np.array(outcome_rep)
    outcome_dem_percent = outcome_dem/(outcome_dem+outcome_rep)
    
    return outcome_dem_percent

def get_mean_dem_vote_percent(graphs):
    total_votes = np.array(get_dem_vote_percent(graphs[0],18))
    for graph in graphs[1:]:
        total_votes += np.array(get_dem_vote_percent(graph,18))
    total_votes = total_votes/len(graphs)
    return(total_votes)

def get_gerry_index(graph, n_district, mean_dem_vote_percent):
    dem_percents = np.array(get_dem_vote_percent(graph, n_district))
    dem_percents = (mean_dem_vote_percent - dem_percents)**2
    gerry_index = (np.sum(dem_percents))**0.5
    return(gerry_index)

def get_graphs(directory):
    graphs = []
    for filename in os.listdir(directory):
        if filename.endswith(".pickle"):
            graph_to_add = nx.read_gpickle(directory+filename)
            #print(type(graph_to_add))
            graphs.append(graph_to_add)
    return(graphs)

def get_all_gerry_indices(directory = 'plans/', n_district = 18):
    graphs = get_graphs(directory)
    mean_dem_vote_percent = get_mean_dem_vote_percent(graphs)
    gerry_indices = []
    for graph in graphs:
        gi = get_gerry_index(graph, n_district, mean_dem_vote_percent)
        gerry_indices.append(gi)
    return(gerry_indices)
    
        

In [124]:
get_mean_dem_vote_percent([example_dict, example_dict2])

array([ 0.78601022,  0.8904682 ,  0.47341489,  0.41713485,  0.4708822 ,
        0.48360544,  0.48978122,  0.51437583,  0.4505124 ,  0.40410325,
        0.46249529,  0.48369063,  0.64531186,  0.69817579,  0.49798434,
        0.43879098,  0.56670758,  0.48553517])

In [125]:
get_gerry_index(example_dict,18,mean_dem_votes)

0.0025391374017133123

In [126]:
gerry_indices = get_all_gerry_indices()
gerry_indices

KeyboardInterrupt: 